In [6]:
import nbformat, os, re

def pre_validate_cnn(notebook_path, folder_name):
    try:
        nb = nbformat.read(notebook_path, as_version=4)
    except Exception:
        return ("FAIL", "Error or corrupted notebook - cannot be opened", 0)

    # BITS ID from filename
    bits_id_from_filename = os.path.basename(notebook_path).split("_")[0]

    # BITS ID from notebook (search all cell types: RNN uses code-cell docstrings)
    bits_id_from_notebook = None
    for cell in nb.cells:
        match = re.search(r"202\d[A-Z]{2}\d{5}", cell.source.upper())
        if match:
            bits_id_from_notebook = match.group(0)
            break

    if not bits_id_from_notebook:
        return ("FAIL", "BITS ID not found in notebook", 0)

    if bits_id_from_filename.upper() != bits_id_from_notebook:
        return ("FAIL", "Filename does not match BITS ID inside notebook", 0)

    # Student name (search all cell types: RNN uses code-cell docstrings)
    student_name = None
    for cell in nb.cells:
        for line in cell.source.split("\n"):
            if line.lower().strip().startswith("name"):
                student_name = line.split(":", 1)[1].strip()
                break
        if student_name:
            break

    if folder_name.lower() != student_name.lower():
        print(student_name.lower())
        print(folder_name.lower())
        return ("FAIL", "Folder name does not match student name in notebook", 0)

    # Execution check
    if not any(c.cell_type == "code" and c.get("outputs") for c in nb.cells):
        return ("FAIL", "All outputs cleared - notebook not executed", 0)

    for cell in nb.cells:
        for out in cell.get("outputs", []):
            if out.output_type == "error":
                return ("FAIL", "Notebook contains execution errors", 0)

    return ("PASS", "Pre-validation successful", "Proceed to grading")


In [8]:
# Input: RNN assignment notebook (run from project root: bits-mtech-assignment)
notebook_path = "2025AA05387_RNN_assignment.ipynb"
folder_name = "SHELAR SACHIN KRISHNA"  # must match Name: in notebook for validation

pre_validate_cnn(notebook_path, folder_name)

('PASS', 'Pre-validation successful', 'Proceed to grading')

In [9]:
import nbformat
import os, re, json

# ============================================================
# ------------------- HELPER FUNCTIONS -----------------------
# ============================================================

def extract_all_source_code(nb):
    return "\n".join(
        cell.source for cell in nb.cells if cell.cell_type == "code"
    )

def extract_json_output(nb):
    for cell in nb.cells:
        if cell.cell_type == "code":
            for out in cell.get("outputs", []):
                if isinstance(out, dict) and "text" in out:
                    try:
                        return json.loads(out["text"])
                    except:
                        pass
    return None

def extract_analysis_from_markdown(nb):
    text = ""
    for cell in nb.cells:
        if cell.cell_type == "markdown":
            text += " " + cell.source
    return text

def count_words(text):
    return len(text.split())

def extract_numbers(text):
    return [int(x) for x in re.findall(r"\d+", text)]

# ============================================================
# --------------- PRE_VALIDATION_CHECK_CNN -------------------
# ============================================================

def pre_validate_cnn(notebook_path, folder_name):
    try:
        nb = nbformat.read(notebook_path, as_version=4)
    except Exception:
        return False, "Error or corrupted notebook - cannot be opened", 0

    # BITS ID from filename
    bits_id_filename = os.path.basename(notebook_path).split("_")[0]

    # BITS ID from notebook (search all cell types: RNN uses code-cell docstrings)
    bits_id_notebook = None
    for cell in nb.cells:
        match = re.search(r"202\d[A-Z]{2}\d{5}", cell.source.upper())
        if match:
            bits_id_notebook = match.group(0)
            break

    if not bits_id_notebook:
        return False, "BITS ID not found in notebook", 0

    if bits_id_filename.upper() != bits_id_notebook:
        return False, "Filename does not match BITS ID inside notebook", 0

    # Student name from notebook (search all cell types)
    student_name = None
    for cell in nb.cells:
        for line in cell.source.split("\n"):
            if line.lower().strip().startswith("name"):
                student_name = line.split(":", 1)[1].strip()
                break
        if student_name:
            break

    if not student_name or folder_name.lower() != student_name.lower():
        return False, "Folder name does not match student name in notebook", 0

    # Execution check
    if not any(c.cell_type == "code" and c.get("outputs") for c in nb.cells):
        return False, "All outputs cleared - notebook not executed", 0

    # Error check
    for cell in nb.cells:
        for out in cell.get("outputs", []):
            if out.get("output_type") == "error":
                return False, "Notebook contains execution errors", 0

    return True, "Pre-validation successful", nb

# ============================================================
# ----------------- CNN_STRICT_GRADING ------------------------
# ============================================================

def CNN_STRICT_GRADING(nb):
    total_marks = 0
    breakdown = {}
    comments = []

    source_code = extract_all_source_code(nb)
    json_data = extract_json_output(nb)

    if not json_data:
        comments.append("No JSON output found")
        return 0, breakdown, comments

    custom = json_data.get("custom_cnn", {})
    tl = json_data.get("transfer_learning", {})
    metrics = ["accuracy", "precision", "recall", "f1_score"]

    # ---------- SECTION 1: Custom CNN (5) ----------
    score = 0
    has_conv = "Conv2D" in source_code or "nn.Conv2d" in source_code
    has_gap = (
        "GlobalAveragePooling" in source_code
        or "AdaptiveAvgPool" in source_code
        or custom.get("architecture", {}).get("has_global_average_pooling") is True
    )
    uses_flatten_dense = (
        ("Flatten" in source_code or "flatten" in source_code)
        and ("Dense" in source_code or "Linear" in source_code)
        and not has_gap
    )

    if has_conv and has_gap and not uses_flatten_dense:
        score += 2
    else:
        comments.append("Custom CNN architecture incorrect or missing GAP")

    framework = custom.get("framework", "").lower()
    if framework in ["keras", "tensorflow"] and "compile(" in source_code:
        score += 1
    elif framework == "pytorch" and "optimizer" in source_code and "criterion" in source_code:
        score += 1
    else:
        comments.append("Custom CNN not properly configured")

    if custom.get("initial_loss") and custom.get("final_loss"):
        score += 1
    else:
        comments.append("Custom CNN loss values missing")

    if sum(1 for m in metrics if custom.get(m)) == 4:
        score += 1
    else:
        comments.append("Custom CNN metrics incomplete")

    breakdown["custom_cnn"] = score
    total_marks += score

    # ---------- SECTION 2: Transfer Learning (5) ----------
    score = 0
    base_model = tl.get("base_model", "").lower()
    valid_models = ["resnet18", "resnet50", "vgg16", "vgg19"]

    if any(m in base_model for m in valid_models) and tl.get("frozen_layers", 0) > 0:
        score += 2
    else:
        comments.append("Transfer learning base model or freezing incorrect")

    if tl.get("has_global_average_pooling") is True:
        score += 1
    else:
        comments.append("Transfer learning missing GAP")

    if tl.get("initial_loss") and tl.get("final_loss"):
        score += 1
    else:
        comments.append("Transfer learning loss missing")

    if sum(1 for m in metrics if tl.get(m)) == 4:
        score += 1
    else:
        comments.append("Transfer learning metrics incomplete")

    breakdown["transfer_learning"] = score
    total_marks += score

    # ---------- SECTION 3: Loss Convergence (4) ----------
    def convergence(init, final):
        if init and final and final < init:
            pct = ((init - final) / init) * 100
            if pct >= 50: return 2
            if pct >= 20: return 1
        return 0

    score = convergence(custom.get("initial_loss"), custom.get("final_loss")) + \
            convergence(tl.get("initial_loss"), tl.get("final_loss"))

    breakdown["training_process"] = score
    total_marks += score

    # ---------- SECTION 4: Metrics Validation (2) ----------
    def valid_metrics(data):
        return all(0 <= data.get(m, -1) <= 1 for m in metrics)

    cnn_ok = valid_metrics(custom)
    tl_ok = valid_metrics(tl)

    score = 2 if cnn_ok and tl_ok else 1 if cnn_ok or tl_ok else 0
    if score == 0:
        comments.append("Metrics missing or invalid")

    breakdown["metrics"] = score
    total_marks += score

    # ---------- SECTION 5: Analysis (2) ----------
    score = 0
    analysis = json_data.get("analysis") or extract_analysis_from_markdown(nb)
    if count_words(analysis) > 200:
        comments.append("Warning: Analysis exceeds 200 words")

    keywords = [
        "performance", "accuracy", "precision", "recall", "f1",
        "transfer", "pretrained", "gap", "overfitting",
        "convergence", "loss", "computational"
    ]
    covered = sum(1 for k in keywords if k in analysis.lower())
    score = 2 if covered >= 8 else 1 if covered >= 5 else 0
    if score == 0:
        comments.append("Analysis lacks depth")

    breakdown["analysis"] = score
    total_marks += score

    # ---------- SECTION 6: Code Structure (2) ----------
    score = 0
    if has_conv and ("ResNet" in source_code or "VGG" in source_code):
        score += 1
    if "custom_cnn" in json_data and "transfer_learning" in json_data:
        score += 1

    breakdown["code_structure"] = score
    total_marks += score

    return total_marks, breakdown, comments

# ============================================================
# --------------------- FINAL DRIVER --------------------------
# ============================================================

def AUTO_GRADE_CNN(notebook_path, folder_name):
    ok, msg, result = pre_validate_cnn(notebook_path, folder_name)

    if not ok:
        return {
            "status": "FAIL",
            "reason": msg,
            "total_marks": 0
        }

    total, breakdown, comments = CNN_STRICT_GRADING(result)

    return {
        "status": "PASS",
        "message": msg,
        "total_marks": total,
        "breakdown": breakdown,
        "comments": comments
    }


In [10]:
# Grade RNN assignment (run from project root so path is valid)
AUTO_GRADE_CNN(
    notebook_path="2025aa05387_RNN_assignment.ipynb",
    folder_name="SHELAR SACHIN KRISHNA"
)


{'status': 'PASS',
 'message': 'Pre-validation successful',
 'total_marks': 0,
 'breakdown': {},
 'comments': ['No JSON output found']}